In [66]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [67]:
import networkx as nx
import numpy as np
def construct_adjlist():
    g = nx.barabasi_albert_graph(n=500, m=3,seed=2017)

    adjlist = nx.generate_adjlist(g, delimiter=",")
    adjlist = [np.array([eval(x)]).flatten() for x in adjlist]

    lens = [len(x) for x in adjlist]
    maxlen = max(lens)
    n = g.number_of_nodes()

    adjlist_m = np.ones([n, maxlen], dtype = np.int32) * (-1)

    for nn in range(n):
        adjlist_m[nn, :lens[nn]] = adjlist[nn]

    return adjlist_m

adjlist_m = construct_adjlist()

In [68]:
%%cython 
import numpy as np
cimport numpy as np

np.random.seed(2017)

cdef _info_spread(np.ndarray[np.int32_t, ndim=2] adjlist_m, \
                np.ndarray[np.int32_t, ndim=1] states):  
    cdef int N = adjlist_m.shape[0]
    cdef int change = 0
    cdef float alpha = 0.4
    cdef float p = 0.2
    cdef float lamda = 0.2
    cdef Py_ssize_t n_i,n_j
    cdef np.ndarray[np.int32_t, ndim=1] new_states = states
    
    for n_i in range(N):
        # n_i is not 'S'
        if states[n_i] == 2 or states[n_i] == 0:
            continue 
            
        # n_i is 'S'
        for n_j in adjlist_m[n_i]: 
            # n_j is 'I'
            if states[n_j] == 0:
                # I-->S
                if np.random.random() < alpha*p:
                    new_states[n_j] = 1
                    change += 1
                # I-->R
                elif np.random.random() < alpha*(1-p):
                    new_states[n_j] = 2
                    change += 1
            # n_j is 'R' or 'S'
            elif np.random.random() < lamda:
                    # S -> R
                    new_states[n_i] = 2
                    change += 1
            
    return new_states,change


def InfoSpread(np.ndarray[np.int32_t, ndim = 2]adjlist_m):
    cdef int N = adjlist_m.shape[0]
    cdef np.ndarray[np.int32_t, ndim = 1] states = np.zeros(N,dtype=np.int32)
    cdef int change = 1
    # set Ignorants and seeds
    states[0:5] = 1
    while change:
        states,change = _info_spread(adjlist_m, states)
    return states

In [70]:
final_states = InfoSpread(adjlist_m)

final_density_stifler = sum(np.where(np.array(final_states) == 2, 1, 0))

0


In [65]:
import numpy as np

a = np.array([1,2,3,3,4,5])

print(np.where(a==3,1,0))

[0 0 1 1 0 0]
